In [1]:
%matplotlib inline
## imports

# site
from scipy.ndimage.morphology import binary_erosion
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import rasterio
import flopy

# std
from pathlib import Path
import yaml

In [2]:
def read_array(rasterfile, masked=True,  band=1):
    with rasterio.open(rasterfile) as src:
        return src.read(band, masked=masked)

In [3]:
def read_profile(rasterfile):
    with rasterio.open(rasterfile) as src:
        return src.profile

In [4]:
def write_array(rasterfile, values, profile):
    with rasterio.open(rasterfile, 'w', **profile) as dst:
        return dst.write(values, 1)

In [5]:
def read_3d_array(rasterfiles, stack_axis=0, masked=True):
    arrays = []
    for rasterfile in rasterfiles:
        arrays.append(read_array(rasterfile, masked=masked))
    return np.ma.stack(arrays, axis=stack_axis)

In [6]:
def save_int(datfile, array):
    save_array(datfile, array, fmt='%i', delimiter=' ')


In [7]:
def save_float(datfile, array):
    save_array(datfile, array, fmt='%15.6E')

In [8]:
def save_array(datfile, array, fmt, delimiter=''):
    np.savetxt(datfile, array, fmt=fmt, delimiter=delimiter)

In [9]:
## input
# name
name = 'mf6brabant_ext'

# workspace
workspace = Path(r'..\output\mf6brabant_using_external_files')

# exe name
exe_name = Path(r'..\bin\mf6.0.3\bin\mf6.exe')

# spatial reference
xllcorner = 60_000.
yllcorner = 322_500.

# grid dimensions
nlay = 19
nrow = 450
ncol = 601
delr = 250.
delc = 250.

# data files
topfile = Path(r'..\data\topbot\RL{ilay:d}.tif')
botfile = Path(r'..\data\topbot\TH{ilay:d}.tif')
idomainfile = Path(r'..\data\boundary\ibound.tif')
               
kdfile = Path(r'..\data\kdc\TX{ilay:d}.tif')
cfile = Path(r'..\data\kdc\CL{ilay:d}.tif')
             
startfile = Path(r'..\data\startingheads\HH{ilay:d}.tif')

rechargefile = Path(r'..\data\recharge\RP1.tif')

drn2005file = Path(r'..\data\mf2005\drn_data.csv')
ghb2005file = Path(r'..\data\mf2005\ghb_data.csv')
riv2005file = Path(r'..\data\mf2005\riv_data.csv')

sqfile = Path(r'..\data\wells\sq_list.csv')

In [10]:
# create workspace directory
workspace.mkdir(exist_ok=True)

In [11]:
# create data directory
datafolder = workspace / 'data'
datafolder.mkdir(exist_ok=True)

In [12]:
# Create the Flopy simulation object
sim = flopy.mf6.MFSimulation(
    sim_name=name,
    exe_name=str(exe_name), 
    version='mf6',
    sim_ws=str(workspace))

# Create the Flopy temporal discretization object
tdis = flopy.mf6.modflow.mftdis.ModflowTdis(sim,
    pname='tdis',
    time_units='DAYS',
    nper=1, 
    perioddata=[(1.0, 1, 1.0)],
    )

# Create the Flopy groundwater flow (gwf) model object
model_nam_file = '{}.nam'.format(name)
gwf = flopy.mf6.ModflowGwf(sim,
    modelname=name, 
    model_nam_file=model_nam_file,
    save_flows=True,
    )

Directory structure already exists for simulation path C:\dev\mf6brabant\mf6brabant\notebooks\..\output\mf6brabant_using_external_files


In [13]:
# read tops
topfiles = (topfile.parent / topfile.name.format(ilay=i + 1) for i in range(nlay))

tops = read_3d_array(topfiles)

# mask bad nodata values
tops = np.ma.masked_where(tops.mask | (tops < -9990.), tops)

In [14]:
# read bots
botfiles = (botfile.parent / botfile.name.format(ilay=i + 1) for i in range(nlay))

bots = read_3d_array(botfiles)

# mask bad nodata values
bots = np.ma.masked_where(bots.mask | (bots < -9990.), bots)

In [15]:
# convert to top, botm
top = tops[0, :, :].filled(0.)
topdatfile = datafolder / 'top.dat'
save_float(topdatfile, top)
top_ext = {'filename': str(topdatfile.relative_to(workspace))}
botm_ext = []
for ilay in range(nlay):
    botm = bots[ilay, :, :].filled((2*ilay + 1) * -1e-3)
    botmdatfile = datafolder / 'botm_l{ilay:02d}.dat'.format(ilay=ilay*2 + 1)
    save_float(botmdatfile, botm)
    botm_ext.append({'filename': str(botmdatfile.relative_to(workspace))})
    if (ilay + 1) < nlay:
        botm = tops[ilay + 1, :, :].filled((2*ilay + 2) * -1e-3)
        botmdatfile = datafolder / 'botm_l{ilay:02d}.dat'.format(ilay=ilay*2 + 2)
        save_float(botmdatfile, botm)
        botm_ext.append({'filename': str(botmdatfile.relative_to(workspace))})

In [16]:
# read idomain
idomain = read_array(idomainfile).filled(0.).astype(np.int)
idomaindatfile = datafolder / 'idomain.dat'
save_int(idomaindatfile, idomain)
idomain_ext = [{'filename': str(idomaindatfile.relative_to(workspace))} for i in range(nlay*2 - 1)]

In [17]:
# initialize the DIS package
dis = flopy.mf6.modflow.mfgwfdis.ModflowGwfdis(gwf,
    pname='dis', nlay=(nlay*2 - 1),
    nrow=nrow, ncol=ncol,
    delr=delr, delc=delc,
    top=top_ext, botm=botm_ext,
    idomain=idomain_ext,
    length_units='METERS',
    )

In [18]:
# read kD
kdfiles = (kdfile.parent / kdfile.name.format(ilay=i + 1) for i in range(nlay))
kd = read_3d_array(kdfiles)

# convert to kh
kh = (kd / (tops - bots))

# fill with low value
kh = kh.filled(1e-6)

# write to file
kh_ext = []
for ilay in range(nlay):
    khdatfile = datafolder / 'kh_l{ilay:02d}.dat'.format(ilay=ilay*2 + 1)
    save_float(khdatfile, kh[ilay, :, :])
    kh_ext.append(
        {'filename': str(khdatfile.relative_to(workspace))},
        )
    if (ilay + 1) < nlay:
        # dummy values
        kh_ext.append(1e-6)

In [19]:
# read c
cfiles = (cfile.parent / cfile.name.format(ilay=i + 1) for i in range(nlay - 1))
c = read_3d_array(cfiles)

# convert to kv
kv = (bots[:-1, :, :] - tops[1:, :, :]) / c

# fill with high value
kv = kv.filled(1e6)

# write to file
kv_ext = []
for ilay in range(nlay):
    # dummy values
    kv_ext.append(1e6)
    if (ilay + 1) < nlay:
        kvdatfile = datafolder / 'kv_l{ilay:02d}.dat'.format(ilay=ilay*2 + 2)
        save_float(kvdatfile, kv[ilay, :, :])
        kv_ext.append(
            {'filename': str(kvdatfile.relative_to(workspace))},
            )

In [20]:
# initialize the NPF package
npf = flopy.mf6.modflow.mfgwfnpf.ModflowGwfnpf(
    model=gwf,
    k=kh_ext,
    k22=kh_ext,
    k33=kv_ext,
)

In [21]:
# read start
startfiles = (startfile.parent / startfile.name.format(ilay=i + 1) for i in range(nlay))
start = read_3d_array(startfiles)

# mask values larger than 1000
start = np.ma.masked_where(start.mask | (start > 1e3), start)

# fill masked with zeros
start = start.filled(0.)

start_ext = []
for ilay in range(nlay):
    startdatfile = datafolder / 'start_l{ilay:02d}.dat'.format(ilay=ilay*2 + 1)
    save_float(startdatfile, start[ilay, :, :])
    start_ext.append(
        {'filename': str(startdatfile.relative_to(workspace))},
        )
    if (ilay + 1) < nlay:
        start_ext.append(
            {'filename': str(startdatfile.relative_to(workspace))},
            )

In [22]:
# Create the initial conditions package
ic = flopy.mf6.modflow.mfgwfic.ModflowGwfic(gwf, pname='ic', strt=start_ext)

In [23]:
# get boundary data from idomain and starting heads
structure = np.ones((3, 3))
isboundary = (idomain == 1) & ~binary_erosion(idomain, structure)
chd_data = []
for ilay in range(dis.nlay.get_data()):    
    for row, col in np.ndindex(*isboundary.shape):
        if isboundary[row, col]:
            chd_data.append(((ilay, row, col), start[ilay//2, row, col]))

chddatfile = datafolder / 'chd.dat'
chd_ext = {0: {
    'filename': str(chddatfile.relative_to(workspace)),
    'data': chd_data,
    }}

In [24]:
# initialize the CHD package
chd = flopy.mf6.modflow.mfgwfchd.ModflowGwfchd(gwf,
    pname='chd',
    maxbound=len(chd_data),
    stress_period_data=chd_ext,
    )

In [25]:
# recharge
recharge = read_array(rechargefile)
rechargedatfile = datafolder / 'recharge.dat'
save_float(rechargedatfile, recharge)
recharge_ext = [{'filename': str(rechargedatfile.relative_to(workspace))}]

In [26]:
# initialize the RCH package
rch = flopy.mf6.ModflowGwfrcha(gwf, pname='rch', recharge=recharge_ext)

In [32]:
# DRN stress period data from Modflow 2005
drn2005_data = pd.read_csv(drn2005file)

# select active cells
is_active = (idomain == 1)[drn2005_data.loc[:, 'i'], drn2005_data.loc[:, 'j']]

get_stress_record = lambda r: (
    (int(r.loc['k']) * 2, int(r.loc['i']), int(r.loc['j'])), r.loc['elev0'], r.loc['cond0']
    )
drn_data = drn2005_data.iloc[is_active, :].apply(get_stress_record, axis=1).values.tolist()

drndatfile = datafolder / 'drn.dat'
drn_ext = {0: {
    'filename': str(drndatfile.relative_to(workspace)),
    'data': drn_data,
    }}

# initialize the DRN package
drn = flopy.mf6.modflow.mfgwfdrn.ModflowGwfdrn(gwf,
    pname='drn',
    maxbound=len(drn_data),
    stress_period_data=drn_ext,
    )

In [33]:
# GHB stress period data from Modflow 2005
ghb2005_data = pd.read_csv(ghb2005file)

# select active cells
is_active = (idomain == 1)[ghb2005_data.loc[:, 'i'], ghb2005_data.loc[:, 'j']]

get_stress_record = lambda r: (
    (int(r.loc['k']) * 2, int(r.loc['i']), int(r.loc['j'])), r.loc['bhead0'], r.loc['cond0']
    )
ghb_data = ghb2005_data.iloc[is_active, :].apply(get_stress_record, axis=1).values.tolist()

ghbdatfile = datafolder / 'ghb.dat'
ghb_ext = {0: {
    'filename': str(ghbdatfile.relative_to(workspace)),
    'data': ghb_data,
    }}

# initialize the GHB package
ghb = flopy.mf6.modflow.mfgwfghb.ModflowGwfghb(gwf,
    pname='ghb',
    maxbound=len(ghb_data),
    stress_period_data=ghb_ext,
    )

In [34]:
# RIV stress period data from Modflow 2005
riv2005_data = pd.read_csv(riv2005file)

# select active cells
is_active = (idomain == 1)[riv2005_data.loc[:, 'i'], riv2005_data.loc[:, 'j']]

get_stress_record = lambda r: (
    (int(r.loc['k']) * 2, int(r.loc['i']), int(r.loc['j'])), r.loc['stage0'], r.loc['cond0'], r.loc['rbot0']
    )
riv_data = riv2005_data.iloc[is_active, :].apply(get_stress_record, axis=1).values.tolist()

rivdatfile = datafolder / 'riv.dat'
riv_ext = {0: {
    'filename': str(rivdatfile.relative_to(workspace)),
    'data': riv_data,
    }}

# initialize the RIV package
riv = flopy.mf6.modflow.mfgwfriv.ModflowGwfriv(gwf,
    pname='riv',
    maxbound=len(riv_data),
    stress_period_data=riv_ext,
    )

In [35]:
# read sources data from csv file
sqs = pd.read_csv(sqfile)

# to row,col from x,y
fwd = rasterio.transform.from_origin(xllcorner, yllcorner + nrow*delr, delc, delr)

# transform xy to row,col
sqs.loc[:, 'row'], sqs.loc[:, 'col'] = (
    rasterio.transform.rowcol(fwd, sqs['xcoordinate'], sqs['ycoordinate'])
    )

# select active cells
is_active = (idomain == 1)[sqs.loc[:, 'row'], sqs.loc[:, 'col']]

# transform xy to row,col
rows, cols = rasterio.transform.rowcol(fwd, sqs['xcoordinate'], sqs['ycoordinate'])

# layer numbers & pumping rates
layernumbers = sqs.loc[:, 'ilay'].tolist()
pumping_rates = sqs.loc[:, 'q_assigned'].tolist()

# convert source data to stress period data records
wel_data = []
for lay, row, col, q in zip(layernumbers, rows, cols, pumping_rates):
    if idomain[row, col]:
        wel_data.append((((lay - 1)*2, row, col), q))

weldatfile = datafolder / 'wel.dat'
wel_ext = {0: {
    'filename': str(weldatfile.relative_to(workspace)),
    'data': wel_data,
    }}

In [36]:
# initialize WEL package
wel = flopy.mf6.modflow.mfgwfwel.ModflowGwfwel(gwf,
    pname='wel',
    maxbound=len(wel_data),
    stress_period_data=wel_ext,
    )

In [37]:
# Create the Flopy iterative model solver (ims) Package object
ims = flopy.mf6.modflow.mfims.ModflowIms(sim, pname='ims', complexity='SIMPLE')

In [38]:
# Create the output control package
headfile = '{}.hds'.format(name)
head_filerecord = [headfile]
budgetfile = '{}.cbb'.format(name)
budget_filerecord = [budgetfile]
saverecord = [('HEAD', 'ALL'), 
              ('BUDGET', 'ALL')]
printrecord = [('HEAD', 'LAST')]
oc = flopy.mf6.modflow.mfgwfoc.ModflowGwfoc(gwf, pname='oc', saverecord=saverecord, 
                                            head_filerecord=head_filerecord,
                                            budget_filerecord=budget_filerecord,
                                            printrecord=printrecord)

In [42]:
# write simulation to new location
sim.write_simulation()

writing simulation...
  writing simulation name file...
  writing simulation tdis package...
  writing ims package ims...
  writing model mf6brabant_ext...
    writing model name file...
    writing package dis...
    writing package npf...
    writing package ic...
    writing package chd...
    writing package rch...
    writing package drn...
    writing package ghb...
    writing package riv...
    writing package wel...
    writing package oc...


In [ ]:
# Run the simulation
success, buff = sim.run_simulation()
print('\nSuccess is: ', success)

FloPy is using the following  executable to run the model: ..\bin\mf6.0.3\bin\mf6.exe
                                   MODFLOW 6
                U.S. GEOLOGICAL SURVEY MODULAR HYDROLOGIC MODEL
                            VERSION 6.0.3 08/09/2018

   MODFLOW 6 compiled Aug 09 2018 13:40:32 with IFORT compiler (ver. 18.0.3)

This software has been approved for release by the U.S. Geological 
Survey (USGS). Although the software has been subjected to rigorous 
review, the USGS reserves the right to update the software as needed 
pursuant to further analysis and review. No warranty, expressed or 
implied, is made by the USGS or the U.S. Government as to the 
functionality of the software and related material nor shall the 
fact of release constitute any such warranty. Furthermore, the 
software is released on condition that neither the USGS nor the U.S. 
Government shall be held liable for any damages resulting from its 
authorized or unauthorized use. Also refer to the USGS Water 
Resou

In [ ]:
# read heads
headfile = workspace / '{name:}.hds'.format(name=name)
hds = flopy.utils.binaryfile.HeadFile(headfile)

In [ ]:
# export to raster
rasterfolder = workspace / 'heads'
rasterfolder.mkdir(exist_ok=True)

profile = read_profile(idomainfile)

heads = hds.get_data()
for i, raster in enumerate(heads[::2]):  # export only horizontal flow layers i.e. 1, 3, .. 37
    raster[idomain < 1] = profile['nodata']  
    rasterfile = rasterfolder / 'phi{ilay:d}.tif'.format(ilay=i + 1)
    write_array(rasterfile, raster, profile)